In [16]:
import numpy as np
import statsmodels.api as sm

np.random.seed(42)

# Simulate the data
def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

# Run simulation to compute probability of detecting a nonzero effect (t-value > 1.96)
def simulate_regression_probability(A=1, B=1, C=10, D=1000, num_simulations=1000):
    t_values = []
    
    for _ in range(num_simulations):
        Y, X, W = simulate(A, B, C, D)
        X = sm.add_constant(X)  # Add constant (intercept) to the model
        model = sm.OLS(Y, np.column_stack((X, W))).fit()  # Fit model with X and W as predictors
        t_values.append(model.tvalues[1])  # Get t-value for X (1st predictor)
    
    # Calculate the proportion of t-values greater than 1.96 in absolute value
    proportion = np.mean(np.abs(t_values) > 1.96)
    return proportion

# Run the simulation
probability = simulate_regression_probability(A=1, B=1, C=10, D=1000, num_simulations=1000)
probability * 100  # Return as percentage

np.float64(86.6)

In [44]:
from scipy.stats import skew

# Function to compute skewness of the regression coefficients for X
def simulate_regression_skewness(A=1, B=1, C=10, D=1000, num_simulations=1000):
    coefficients = []
    
    for _ in range(num_simulations):
        Y, X, W = simulate(A, B, C, D)
        X = sm.add_constant(X)  # Add constant (intercept) to the model
        model = sm.OLS(Y, np.column_stack((X, W))).fit()  # Fit model with X and W as predictors
        coefficients.append(model.params[1])  # Get coefficient for X (1st predictor)
    
    # Calculate skewness of the coefficients for X
    return skew(coefficients)

# Run the simulation to calculate skewness
skewness = simulate_regression_skewness(A=1, B=1, C=10, D=1000, num_simulations=10000)
skewness  # Return the skewness value

np.float64(-0.003495101516301757)

In [23]:
# Function to find B that results in 50% detection of nonzero coefficient for X
def find_B_for_detection_probability(A=1, C=10, D=1000, target_probability=0.5, B_values=None, num_simulations=1000):
    if B_values is None:
        B_values = [0.2, 0.6, 1.8, 5.4]  # A range of B values to check
        
    probabilities = {}
    
    for B in B_values:
        prob = simulate_regression_probability(A, B, C, D, num_simulations)
        probabilities[B] = prob
        
    closest_B = min(probabilities, key=lambda x: abs(probabilities[x] - target_probability))
    return closest_B, probabilities[closest_B]

# Find the value of B that results in ~50% detection
best_B, detection_probability = find_B_for_detection_probability(A=1, C=10, D=1000, target_probability=0.5)
best_B, detection_probability * 100  # Return the closest B value and the detection probability in percentage

(0.6, np.float64(49.9))

In [24]:
# Function to find A that results in 50% detection of nonzero coefficient for X
def find_A_for_detection_probability(B=1, C=10, D=100, target_probability=0.5, A_values=None, num_simulations=1000):
    if A_values is None:
        A_values = [0.5, 1.0, 2.0, 4.0]  # A range of A values to check
        
    probabilities = {}
    
    for A in A_values:
        prob = simulate_regression_probability(A, B, C, D, num_simulations)
        probabilities[A] = prob
        
    closest_A = min(probabilities, key=lambda x: abs(probabilities[x] - target_probability))
    return closest_A, probabilities[closest_A]

# Find the value of A that results in ~50% detection
best_A, detection_probability = find_A_for_detection_probability(B=1, C=10, D=100, target_probability=0.5)
best_A, detection_probability * 100  # Return the closest A value and the detection probability in percentage

(2.0, np.float64(49.8))